<a href="https://colab.research.google.com/github/Cristina2003leon/master-analitica-datos-cristina-mantilla/blob/main/Cristinadataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [107]:
import sqlite3
import pandas as pd


!rm base_datos.db # Also remove the main db file itself to ensure a clean start

con = sqlite3.connect("base_datos.db", timeout=5)

# Borrar tabla si existe
con.execute("DROP TABLE IF EXISTS animales;")

# Crear tabla
con.execute("""
CREATE TABLE animales(
    id INTEGER PRIMARY KEY,
    Nombre TEXT,
    PeligrosidadVeneno INTEGER CHECK(PeligrosidadVeneno >= 0 AND PeligrosidadVeneno <= 10),
    Habitat TEXT,
    EsperanzaDeVida INTEGER
);
""")

In [108]:
con.executemany("""
INSERT INTO animales (id, Nombre, PeligrosidadVeneno, Habitat, EsperanzaDeVida)
VALUES (?, ?, ?, ?, ?)
""", [
    (1, 'Serpiente Taipán', 10, 'Bosques secos de Australia', 15),
    (2, 'Rana Dardo Dorada', 9, 'Selva tropical', 10),
    (3, 'Viuda Negra', 8, 'Zonas templadas', 3),
    (4, 'Mamba Negra', 9, 'Sabana africana', 20),
    (5, 'Escorpión Amarillo', 7, 'Regiones áridas', 6),
])




In [109]:
con.execute("""
INSERT INTO animales (Nombre, PeligrosidadVeneno, Habitat, EsperanzaDeVida)
VALUES ("Cobra Real", 9, "Selva tropical del sudeste asiático", 20);
""")

In [110]:
pd.read_sql("SELECT * FROM animales;", con)


,id,Nombre,PeligrosidadVeneno,Habitat,EsperanzaDeVida
0,1,Serpiente Taipán,10,Bosques secos de Australia,15
1,2,Rana Dardo Dorada,9,Selva tropical,10
2,3,Viuda Negra,8,Zonas templadas,3
3,4,Mamba Negra,9,Sabana africana,20
4,5,Escorpión Amarillo,7,Regiones áridas,6
5,6,Cobra Real,9,Selva tropical del sudeste asiático,20


In [111]:
con.execute("DROP TABLE IF EXISTS ecologia;")

con.execute("""
CREATE TABLE ecologia(
    id INTEGER,
    Dieta TEXT,
    Region TEXT,
    PoblacionEstimada INTEGER
);
""")


In [112]:
con.executemany("""
INSERT INTO ecologia (id, Dieta, Region, PoblacionEstimada)
VALUES (?, ?, ?, ?)
""", [
    (1, 'Carnívoro', 'Australia', 100000),
    (2, 'Insectívoro', 'Colombia', 5000),
    (3, 'Insectívoro', 'Norteamérica / Europa', 200000),
    (4, 'Carnívoro', 'África', 50000),
    (5, 'Insectívoro', 'Oriente Medio', 800000),
])



In [113]:
con.execute("""
INSERT INTO ecologia (id, Dieta, Region, PoblacionEstimada)
VALUES (6, "Principalmente serpientes (ofiófaga)", "Sudeste Asiático", 30000);
""")


In [114]:
pd.read_sql("SELECT * FROM ecologia;", con)


,id,Dieta,Region,PoblacionEstimada
0,1,Carnívoro,Australia,100000
1,2,Insectívoro,Colombia,5000
2,3,Insectívoro,Norteamérica / Europa,200000
3,4,Carnívoro,África,50000
4,5,Insectívoro,Oriente Medio,800000
5,6,Principalmente serpientes (ofiófaga),Sudeste Asiático,30000


In [115]:
avg = """
SELECT
    A.id,
    A.Nombre,
    A.PeligrosidadVeneno,
    Stats.Peligrosidad_Media,
    A.PeligrosidadVeneno - Stats.Peligrosidad_Media AS Desviacion
FROM animales A,
    (SELECT AVG(PeligrosidadVeneno) AS Peligrosidad_Media FROM animales) Stats;
"""
pd.read_sql(avg, con)


,id,Nombre,PeligrosidadVeneno,Peligrosidad_Media,Desviacion
0,1,Serpiente Taipán,10,8.666667,1.333333
1,2,Rana Dardo Dorada,9,8.666667,0.333333
2,3,Viuda Negra,8,8.666667,-0.666667
3,4,Mamba Negra,9,8.666667,0.333333
4,5,Escorpión Amarillo,7,8.666667,-1.666667
5,6,Cobra Real,9,8.666667,0.333333


In [116]:
juntar = """
SELECT
    a.Nombre,
    a.PeligrosidadVeneno,
    a.Habitat,
    a.EsperanzaDeVida,
    e.Dieta,
    e.Region,
    e.PoblacionEstimada
FROM animales a
INNER JOIN ecologia e
    ON a.id = e.id;
"""

pd.read_sql(juntar, con)


,Nombre,PeligrosidadVeneno,Habitat,EsperanzaDeVida,Dieta,Region,PoblacionEstimada
0,Serpiente Taipán,10,Bosques secos de Australia,15,Carnívoro,Australia,100000
1,Rana Dardo Dorada,9,Selva tropical,10,Insectívoro,Colombia,5000
2,Viuda Negra,8,Zonas templadas,3,Insectívoro,Norteamérica / Europa,200000
3,Mamba Negra,9,Sabana africana,20,Carnívoro,África,50000
4,Escorpión Amarillo,7,Regiones áridas,6,Insectívoro,Oriente Medio,800000
5,Cobra Real,9,Selva tropical del sudeste asiático,20,Principalmente serpientes (ofiófaga),Sudeste Asiático,30000


In [117]:
try:


    #1 Actualizamos la peligrosidad de un animal existente
    con.execute("""
    UPDATE animales
    SET PeligrosidadVeneno = 9
    WHERE Nombre = 'Cobra Real'
    """)

    # 2️ Insertamos un nuevo animal con valores válidos
    con.execute("""
    INSERT INTO animales (Nombre, PeligrosidadVeneno, Habitat, EsperanzaDeVida)
    VALUES (?, ?, ?, ?)
    """, ("Lagarto Azul", 3, "Selva", 12))

    # 3️ Intentamos insertar un animal con datos inválidos
    # (peligrosidad < 0 → viola CHECK si lo tienes definido)
    con.execute("""
    INSERT INTO animales (Nombre, PeligrosidadVeneno, Habitat, EsperanzaDeVida)
    VALUES (?, ?, ?, ?)
    """, ("Dragón Verde", -5, "Bosque", 50))

    # Si todo va bien, confirmamos
    con.commit()

except sqlite3.IntegrityError as e:
    # Si hay error, deshacemos toda la transacción
    print("Error detectado, deshaciendo cambios:", e)
    con.rollback()


Error detectado, deshaciendo cambios: CHECK constraint failed: PeligrosidadVeneno >= 0 AND PeligrosidadVeneno <= 10
